In [1]:
import os 
import subprocess
import pandas as pd
import sys
import hcrfish

# Example: Drosophila Melanogaster

Download Genome

In [ ]:
# Make directory 
output_dir = "input/dmel/genome"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Download the genome
rsync_path = "rsync://hgdownload.soe.ucsc.edu/goldenPath/dm6/bigZips/dm6.fa.gz"
output_path = os.path.join(output_dir, "dm6.fa.gz")
command = f"rsync --partial --progress {rsync_path} {output_path}"
subprocess.run(command, shell=True)

# Unzip the genome
output_path = "input/dmel/genome/dm6.fa.gz"
command = f"gunzip {output_path}"
subprocess.run(command, shell=True)

# Unhide genome file 
output_path = "input/dmel/genome/dm6.fa"
command = f"chflags nohidden {output_path} {output_path}" 
subprocess.run(command, shell=True)

dm6.fa.gz
    45153922 100%  559.25MB/s    0:00:00 (xfer#1, to-check=0/1)

sent 47134 bytes  received 27010 bytes  29657.60 bytes/sec
total size is 45153922  speedup is 609.00


gunzip: raw_data/dmel/genome/dm6.fa already exists -- skipping


CompletedProcess(args='chflags nohidden raw_data/dmel/genome/dm6.fa raw_data/dmel/genome/dm6.fa', returncode=0)

Download Transcriptome

In [ ]:
output_dir = "input/dmel/transcriptome"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Download the transcriptome 
rsync_path = "rsync://hgdownload.soe.ucsc.edu/goldenPath/dm6/bigZips/genes/dm6.ncbiRefSeq.gtf.gz"
output_path = os.path.join(output_dir, "dm6.ncbiRefSeq.gtf.gz")
command = f"rsync --partial --progress {rsync_path} {output_path}"
subprocess.run(command, shell=True)

# Unzip the transcriptome 
output_path = "input/dmel/transcriptome/dm6.ncbiRefSeq.gtf.gz"
command = f"gunzip {output_path}"
subprocess.run(command, shell=True)

# Unhide transcriptome file
output_path = "input/dmel/transcriptome/dm6.ncbiRefSeq.gtf"
command = f"chflags nohidden {output_path}"
subprocess.run(command, shell=True)

dm6.ncbiRefSeq.gtf.gz
     5133989 100%  816.03MB/s    0:00:00 (xfer#1, to-check=0/1)

sent 13646 bytes  received 9182 bytes  15218.67 bytes/sec
total size is 5133989  speedup is 224.90


gunzip: raw_data/dmel/transcriptome/dm6.ncbiRefSeq.gtf already exists -- skipping


CompletedProcess(args='chflags nohidden raw_data/dmel/transcriptome/dm6.ncbiRefSeq.gtf', returncode=0)

Build transcriptome object 

In [ ]:
mel_genome_path = "input/dmel/genome/dm6.fa"
mel_transcriptome_path = "input/dmel/transcriptome/dm6.ncbiRefSeq.gtf"
mel_object_name = "mel_transcriptome"
update_transcriptome_object(mel_genome_path, mel_transcriptome_path, mel_object_name, 'dmel')

Found 17868 unique genes.


100%|██████████| 17868/17868 [00:23<00:00, 746.26it/s] 


Transcriptome(genes=17868)
Transcriptome object has been updated and saved to mel_transcriptome.pkl


In [5]:
tr = load_transcriptome_object(mel_object_name)

In [6]:
check_exons_contain_all_features(tr)

Export mRNA to Fasta file

In [ ]:
# Export mRNA sequences without introns
output_dir = "input/dmel/transcriptome/mRNA_no_introns"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_path = os.path.join(output_dir, "mRNA_no_introns.fasta")

with open(output_path, "w") as output_file:
    for gene_name in tr.genes.keys():
        gene = tr.genes[gene_name]
        for transcript in gene.transcripts:
            output_file.write(f">{transcript.name} gene={gene.name} location={transcript.chromosome}:{transcript.position[0]}-{transcript.position[1]} strand={transcript.strand} \n{transcript.mrna_sequence} \n")


# Export mRNA sequences with introns
output_dir = "input/dmel/transcriptome/mRNA_yes_introns"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_path = os.path.join(output_dir, "mRNA_yes_introns.fasta")

with open(output_path, "w") as output_file:
    for gene_name in tr.genes.keys():
        gene = tr.genes[gene_name]
        for transcript in gene.transcripts:
            output_file.write(f">{transcript.name} gene={gene.name} location={transcript.chromosome}:{transcript.position[0]}-{transcript.position[1]} strand={transcript.strand} \n{transcript.dna_sequence} \n")


Create Blast Databases

In [8]:
# Check that makeblastdb is installed
!makeblastdb -version

makeblastdb: 2.15.0+
 Package: blast 2.15.0, build Oct 19 2023 15:16:13


In [ ]:
# Make blast database without introns 
input_path = "input/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns.fasta"
output_path = "input/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns"
command = f"makeblastdb -in {input_path} -dbtype nucl -parse_seqids -out {output_path}"
subprocess.run(command, shell=True)

# Make blast database with introns
input_path = "input/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns.fasta"
output_path = "input/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns"
command = f"makeblastdb -in {input_path} -dbtype nucl -parse_seqids -out {output_path}"
subprocess.run(command, shell=True)



Building a new DB, current time: 12/02/2024 08:53:02
New DB name:   /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns
New DB title:  raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dmel/transcriptome/mRNA_no_introns/mRNA_no_introns
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 35121 sequences in 0.883459 seconds.




Building a new DB, current time: 12/02/2024 08:53:04
New DB name:   /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns
New DB title:  raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns

CompletedProcess(args='makeblastdb -in raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns.fasta -dbtype nucl -parse_seqids -out raw_data/dmel/transcriptome/mRNA_yes_introns/mRNA_yes_introns', returncode=0)

# Example: Drosophila Yakuba
- https://www.ncbi.nlm.nih.gov/refseq/annotation_euk/Drosophila_yakuba/102/
- ASSEMBLY NAME:	Prin_Dyak_Tai18E2_2.1
- ASSEMBLY ACCESSION:	GCF_016746365.2
- ASSEMBLY SUBMITTER:	Princeton University
- ASSEMBLY DATE:	30 July 2021
- ASSEMBLY TYPE:	Haploid

Download Genome

In [ ]:
# Make directory 
output_dir = "input/dyak/genome"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Download the genome 
rsync_path = "rsync://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/016/746/365/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fna.gz"
output_path = os.path.join(output_dir, "GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fna.gz")
command = f"rsync --partial --progress {rsync_path} {output_path}"
subprocess.run(command, shell=True)


# Unzip the genome
output_path = "input/dyak/genome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fna.gz"
command = f"gunzip {output_path}"
subprocess.run(command, shell=True)

# Unhide genome file 
output_path = "input/dyak/genome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fna"
command = f"chflags nohidden {output_path} {output_path}" 
subprocess.run(command, shell=True)

# convert from fna to fa 
input_path = "input/dyak/genome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fna"
output_path = "input/dyak/genome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fa"
command = f"mv {input_path} {output_path}"
subprocess.run(command, shell=True)


Warning Notice!

You are accessing a U.S. Government information system which includes this
computer, network, and all attached devices. This system is for
Government-authorized use only. Unauthorized use of this system may result in
disciplinary action and civil and criminal penalties. System users have no
expectation of privacy regarding any communications or data processed by this
system. At any time, the government may monitor, record, or seize any
communication or data transiting or stored on this information system.

-------------------------------------------------------------------------------

Welcome to the NCBI rsync server.


GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fna.gz
    45811814 100%   18.69MB/s    0:00:02 (xfer#1, to-check=0/1)

sent 38 bytes  received 45823139 bytes  18329270.80 bytes/sec
total size is 45811814  speedup is 1.00


CompletedProcess(args='mv raw_data/dyak/genome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fna raw_data/dyak/genome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fa', returncode=0)

Download Transcriptome

In [ ]:
output_dir = "input/dyak/transcriptome"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Download the transcriptome 
rsync_path = "rsync://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/016/746/365/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.gtf.gz"
output_path = os.path.join(output_dir, "GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.gtf.gz")
command = f"rsync --partial --progress {rsync_path} {output_path}"
subprocess.run(command, shell=True)

# Unzip the transcriptome 
output_path = "input/dyak/transcriptome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.gtf.gz"
command = f"gunzip {output_path}"
subprocess.run(command, shell=True)

# Unhide transcriptome file
output_path = "input/dyak/transcriptome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.gtf"
command = f"chflags nohidden {output_path}"
subprocess.run(command, shell=True)




Warning Notice!

You are accessing a U.S. Government information system which includes this
computer, network, and all attached devices. This system is for
Government-authorized use only. Unauthorized use of this system may result in
disciplinary action and civil and criminal penalties. System users have no
expectation of privacy regarding any communications or data processed by this
system. At any time, the government may monitor, record, or seize any
communication or data transiting or stored on this information system.

-------------------------------------------------------------------------------

Welcome to the NCBI rsync server.


GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.gtf.gz
     5165818 100%  447.86MB/s    0:00:00 (xfer#1, to-check=0/1)

sent 13682 bytes  received 9237 bytes  15279.33 bytes/sec
total size is 5165818  speedup is 225.39


gunzip: raw_data/dyak/transcriptome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.gtf already exists -- skipping


CompletedProcess(args='chflags nohidden raw_data/dyak/transcriptome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.gtf', returncode=0)

Build transcriptome object 

In [ ]:
yak_genome_path = "input/dyak/genome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.fa"
yak_transcriptome_path = "input/dyak/transcriptome/GCF_016746365.2_Prin_Dyak_Tai18E2_2.1_genomic.gtf"
yak_object_name = "yak_transcriptome"
update_transcriptome_object(yak_genome_path, yak_transcriptome_path, yak_object_name, 'dyak')

Found 16150 unique genes.


100%|██████████| 16150/16150 [00:21<00:00, 762.36it/s]


Transcriptome(genes=16150)
Transcriptome object has been updated and saved to yak_transcriptome.pkl


In [13]:
tr = load_transcriptome_object(yak_object_name)

In [14]:
check_exons_contain_all_features(tr)

Export mRNA to Fasta file

In [ ]:
# Export mRNA without introns
output_dir = "input/dyak/transcriptome/mRNA_no_introns"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_path = os.path.join(output_dir, "mRNA_no_introns.fasta")

with open(output_path, "w") as output_file:
    for gene_name in tr.genes.keys():
        gene = tr.genes[gene_name]
        for transcript in gene.transcripts:
            output_file.write(f">{transcript.name} gene={gene.name} location={transcript.chromosome}:{transcript.position[0]}-{transcript.position[1]} strand={transcript.strand} \n{transcript.mrna_sequence} \n")


# Export mRNA with introns 
output_dir = "input/dyak/transcriptome/mRNA_yes_introns"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_path = os.path.join(output_dir, "mRNA_yes_introns.fasta")

with open(output_path, "w") as output_file:
    for gene_name in tr.genes.keys():
        gene = tr.genes[gene_name]
        for transcript in gene.transcripts:
            output_file.write(f">{transcript.name} gene={gene.name} location={transcript.chromosome}:{transcript.position[0]}-{transcript.position[1]} strand={transcript.strand} \n{transcript.dna_sequence} \n")


Create Blast Databases

In [ ]:
input_path = "input/dyak/transcriptome/mRNA_no_introns/mRNA_no_introns.fasta"
output_path = "input/dyak/transcriptome/mRNA_no_introns/mRNA_no_introns"
command = f"makeblastdb -in {input_path} -dbtype nucl -parse_seqids -out {output_path}"
subprocess.run(command, shell=True)

input_path = "input/dyak/transcriptome/mRNA_yes_introns/mRNA_yes_introns.fasta"
output_path = "input/dyak/transcriptome/mRNA_yes_introns/mRNA_yes_introns"
command = f"makeblastdb -in {input_path} -dbtype nucl -parse_seqids -out {output_path}"
subprocess.run(command, shell=True)



Building a new DB, current time: 12/02/2024 08:53:38
New DB name:   /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dyak/transcriptome/mRNA_no_introns/mRNA_no_introns
New DB title:  raw_data/dyak/transcriptome/mRNA_no_introns/mRNA_no_introns.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dyak/transcriptome/mRNA_no_introns/mRNA_no_introns
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 28247 sequences in 0.8102 seconds.




Building a new DB, current time: 12/02/2024 08:53:39
New DB name:   /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dyak/transcriptome/mRNA_yes_introns/mRNA_yes_introns
New DB title:  raw_data/dyak/transcriptome/mRNA_yes_introns/mRNA_yes_introns.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/giacomo.glotzer/Desktop/Drosophila-HCR/raw_data/dyak/transcriptome/mRNA_yes_introns/mRNA_yes_introns
K

CompletedProcess(args='makeblastdb -in raw_data/dyak/transcriptome/mRNA_yes_introns/mRNA_yes_introns.fasta -dbtype nucl -parse_seqids -out raw_data/dyak/transcriptome/mRNA_yes_introns/mRNA_yes_introns', returncode=0)

Get Chromosomes

In [37]:
# Get the chromosomes and number of genes per chromosome 
chromosomes = {}
for gene_name in tr.genes.keys():
    gene = tr.get_gene(gene_name)
    chromosomes[gene.chromosome] = chromosomes.get(gene.chromosome, 0) + 1
chrom_counts = pd.DataFrame(chromosomes.items(), columns=['Chromosome', 'Number of Genes'])
chrom_counts = chrom_counts.sort_values(by='Number of Genes', ascending=False)
chrom_counts = chrom_counts.reset_index(drop=True)
chrom_counts[0:10]

,Chromosome,Number of Genes
0,NC_052530.2,3822
1,NC_052528.2,3212
2,NC_052527.2,3192
3,NC_052529.2,3068
4,NC_052526.2,2368
5,NC_052531.2,107
6,NW_025048787.1,63
7,NW_025048801.1,42
8,NW_025048817.1,40
9,NC_001322.1,37
